In [ ]:
from IPython.display import display, clear_output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import P7_DataBreed
import p5_util

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from  sklearn import model_selection
import numpy as np


In [ ]:
import p5_util
filename= './data/df_breed_image.dump'
if False :
    df = p5_util.object_load(filename)
    df.shape

In [ ]:
import p5_util
if True :
    oP7_DataBreed = p5_util.object_load('./data/oP7_DataBreed.dump')
    oP7_DataBreed.show()
    oP7_DataBreed.breed_show()

#### Build a sample dataset :

Sampling is leaded by 2 parameters :
* `breed_count` : number of breeds into the dataset
* `image_per_breed_count` : for each breed, number of image into sample.

In [ ]:
import P7_DataBreed

oP7_DataBreed=P7_DataBreed.P7_DataBreed('./data/Images')
oP7_DataBreed.std_size=None
oP7_DataBreed.is_squarred=False
oP7_DataBreed.is_kp_filtered=False
is_splitted=True
oP7_DataBreed.split_ratio=(3,3)
oP7_DataBreed.load()
breed_count=3
image_per_breed_count=100
list_breed_sample=['n02107142-Doberman','n02115641-dingo','n02113978-Mexican_hairless']
#list_breed_sample=['n02107142-Doberman']
oP7_DataBreed.list_breed_sample = list_breed_sample
oP7_DataBreed.sampling(breed_count, image_per_breed_count)

oP7_DataBreed.show()
oP7_DataBreed.show_breed_name()

**Execute following cell in order to update object version with no change in attributes**

* A backup instance of object is created. This backup instance is empty.
* If object instance is not defined then it is read from dmped file.
* object instance is copied back into backup instance
* Backup instance is deleted

In [ ]:
import P7_DataBreed
if False :
    oP7_DataBreed = P7_DataBreed.update_object_save(oP7_DataBreed, is_saved=False,is_new_attribute=True)
    oP7_DataBreed.show()


**Building Data-model**

DataFrame with columns as images, breed names and labels is built.

Labels are issued from encoding breed names. 

Labels are then used for training Keras network. 


In [ ]:
df = oP7_DataBreed.df_build()
df.label.unique(),df.shape

**Dataframe is saved**

In [ ]:
import p5_util

filename= './data/df_breed_image.dump'
p5_util.object_dump(df,filename)
df.sample()

### Image truncated against image resize

In [ ]:
ser_pil_image = df['image']
ser_label = df['label']

#### Original image sample

In [ ]:
pil_image = ser_pil_image.iloc[0]
print(pil_image.size)
pil_image

#### Resized image

Resize does not lead to loss of information due to truncation.
<br>
Resizing allows to consume less resources into CNN network.

In [ ]:
from PIL import Image
pil_image_resize = pil_image.resize((224,224))
pil_image_resize

#### Truncated image as square

In [ ]:
import numpy as np
import P7_DataBreed
if False :
    square=(204,204)
    pil_image_truncated = P7_DataBreed.pil_square(ser_pil_image.iloc[0], square=square)
    print(np.array(pil_image_truncated).shape)
    #arr_keras_X = arr_keras_X.reshape(1,arr_keras_X.shape[0],arr_keras_X.shape[1],arr_keras_X.shape[2])
    pil_image_truncated

### Building train and test sample, Keras compliant

In [ ]:
square = None
resize = (224,224)
test_size=0.1

In [ ]:
import P7_DataBreed

arr_keras_image_train, arr_keras_image_test, arr_label_train, arr_label_test \
= P7_DataBreed.p7_keras_X_train_test_build(ser_pil_image, ser_label, test_size=test_size, square=square, resize=resize)

arr_keras_image_train.shape, arr_keras_image_test.shape, arr_label_train.shape, arr_label_test.shape

#### Dump train and test sample

In [ ]:
import p5_util
filename='./data/arr_keras_X_y_train_test.dump'
p5_util.object_dump((arr_keras_image_train, arr_keras_image_test, arr_label_train, arr_label_test),filename)

# <font color='blus'>1. Breeds classification using Keras MLP model</font>

#### Loading data

In [ ]:
import numpy as np
from keras.utils import to_categorical
import p5_util

filename='./data/arr_keras_X_y_train_test.dump'
(X_train,X_test, y_train, y_test) = p5_util.object_load(filename)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

#### Data normalization
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')

# Scale the data to lie between 0 to 1
X_train /= 255
X_test /= 255
## Retrieve number of classes
nClasses = len(np.unique(y_train))
print("Number of classses= "+str(nClasses))

### One hot encoding of labels
y_train_one_hot_enc = to_categorical(y_train)
y_test_one_hot_enc = to_categorical(y_test)
print()
print(y_train_one_hot_enc.shape)
print(y_test_one_hot_enc.shape)

dimData = np.prod(X_train.shape[1:])
print(dimData)

X_train = X_train.reshape(X_train.shape[0], dimData)
X_test  = X_test.reshape(X_test.shape[0], dimData)
print(X_train.shape)
print(X_test.shape)

#### Building MLP network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras import optimizers

mlp_model = Sequential()
mlp_model.add(Dense(512, activation='relu', input_shape=(dimData,)))
mlp_model.add(Dropout(0.5))

mlp_model.add(Dense(512, activation='relu'))
mlp_model.add(Dropout(0.5))

#mlp_model.add(Dense(512, activation='relu'))
#mlp_model.add(Dropout(0.5))

mlp_model.add(Dense(nClasses, activation='softmax'))
sgd = optimizers.SGD(lr=5.e-4)

mlp_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

mlp_model.summary()

#### Training model

In [ ]:
history = mlp_model.fit(X_train,
                    y_train_one_hot_enc,
                    epochs=50,
                    batch_size=50,
                    validation_data=(X_test,y_test_one_hot_enc))

#### Display results

In [ ]:
import matplotlib.pyplot as plt

[test_loss, test_acc] = mlp_model.evaluate(X_test, y_test_one_hot_enc)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

#Plot the Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=1.0)
plt.plot(history.history['val_loss'],'b',linewidth=1.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves ',fontsize=16)
 
#Plot the Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=1.0)
plt.plot(history.history['val_acc'],'b',linewidth=1.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

# <font color='blus'>2. Breeds classification using CNN VGG16 network</font>

#### Read dumped file for train and test sample

In [ ]:
import p5_util
filename='./data/arr_keras_X_y_train_test.dump'
(arr_X_train,arr_X_test, arr_y_train, arr_y_test) = p5_util.object_load(filename)
print(arr_X_train.shape,arr_X_test.shape,arr_y_train.shape,arr_y_test.shape)

### Building VGG16 Keras layers

#### 1st convolution layer is added 

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout

VGG16Seq = Sequential()  # Création d'un réseau de neurones vide 

# Ajout de la première couche de convolution, suivie d'une couche ReLU
w=arr_X_train.shape[1]
h=arr_X_train.shape[2]
c=arr_X_train.shape[3]
print(w,h,c)
VGG16Seq.add(Conv2D(64, (3, 3), input_shape=(w, h, c), padding='same', activation='relu'))
VGG16Seq.add(Conv2D(64, (3, 3), activation='relu'))
VGG16Seq.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#### 2nd convolution layer is added 

In [ ]:
# Ajout de la deuxième couche de convolution, suivie  d'une couche ReLU
VGG16Seq.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
VGG16Seq.add(Conv2D(64, (3, 3), activation='relu'))
VGG16Seq.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#### 3th convolution layer is added 

In [ ]:
# Ajout de la deuxième couche de convolution, suivie  d'une couche ReLU
VGG16Seq.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
VGG16Seq.add(Conv2D(64, (3, 3), activation='relu'))
VGG16Seq.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#### Pooling layer is added 

In [ ]:
nb_labels = arr_y_train.shape[0]
print("Number of labels for training process : "+str(nb_labels))
VGG16Seq.add(Flatten())  # Conversion des matrices 3D en vecteur 1D

#### 1st Dense layer is added

In [ ]:
VGG16Seq.add(Dense(nb_labels, activation='relu'))
VGG16Seq.add(Dropout(0.5))

#### 2nd Dense layer is added

#### 3th Dense layer is added

In [ ]:
import numpy as np
# Ajout de la dernière couche fully-connected qui permet de classifier
VGG16Seq.add(Dense(nb_labels, activation='softmax'))

#### VGG16 compilation

In [ ]:
from keras import optimizers
sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = optimizers.RMSprop(lr=1e-4)

#my_VGG16.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
#my_VGG16.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
VGG16Seq.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


#### Keras Sequential model is dumped

In [ ]:
import p5_util
p5_util.object_dump(VGG16Seq,'./data/VGG16Seq_sgd.dump')

In [ ]:
VGG16Seq.summary()

## Loading VGG166 Sequential model

In [ ]:
from IPython.display import display, clear_output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import P7_DataBreed
import p5_util

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from  sklearn import model_selection
import numpy as np


In [ ]:
import p5_util
if True:
    VGG16Seq = p5_util.object_load('./data/VGG16Seq_sgd.dump')
    VGG16Seq.summary()

import p5_util
import P7_DataBreed
if False :
    oP7_DataBreed = p5_util.object_load('./data/oP7_DataBreed.dump')
    oP7_DataBreed.show()
    oP7_DataBreed.breed_show()


## Test for images classification

#### Data is loaded form dumped file

In [ ]:
import p5_util
filename='./data/arr_keras_X_y_train_test.dump'
(arr_X_train,arr_X_test, arr_y_train, arr_y_test) = p5_util.object_load(filename)
print(arr_X_train.shape,arr_X_test.shape,arr_y_train.shape,arr_y_test.shape)

#### Data normalization

arr_X_train = arr_X_train.astype('float32')
arr_X_test = arr_X_test.astype('float32')

# Scale the data to lie between 0 to 1
arr_X_train /= 255
arr_X_test /= 255

#### Model training

In [ ]:
print(arr_X_train.shape)
history = VGG16Seq.fit(arr_X_train, arr_y_train, epochs=50, batch_size=90,verbose=1,validation_data=(arr_X_test, arr_y_test))

In [ ]:
[test_loss, test_acc] = VGG16Seq.evaluate(arr_X_test, arr_y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

In [ ]:
#Plot the Loss Curves
import matplotlib.pyplot as plt

plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=1.0)
plt.plot(history.history['val_loss'],'b',linewidth=1.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves ',fontsize=16)
 
#Plot the Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=1.0)
plt.plot(history.history['val_acc'],'b',linewidth=1.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
import p5_util
is_score_dumped=True
if is_score_dumped is True :
    filename = './data/dict_cls_score.dump'
    dict_cls_score = p5_util.object_load(filename)
else:
    dict_cls_score = dict()
dict_classifier = dict()

dict_cls_score

In [ ]:
y_pred = VGG16Seq.predict(arr_X_test)

In [ ]:
filename= './data/result_VGG16Seq.dump'
p5_util.object_dump(y_pred,filename)

#### Measure of accuracy

Result is stored into `arr_result` where :
* rows : number of observations from test dataset
* columns : probabibilty for each observation from train dataset.

For each row from `arr_result`, greater probability value is searched for. 
<br>
The result of search provides index of label into `list_index`.


In [ ]:
import p5_util
filename= './data/result_vgg16.dump'
p5_util.object_load(y_pred,filename)

In [ ]:
(loss, score_vgg16_seq) = VGG16Seq.evaluate(arr_X_test, arr_y_test, verbose=True)

score_vgg16_seq

In [ ]:
import pandas as pd
import p5_util
import p6_util_plot
is_dumped=True
if is_dumped is True:
    filename = './data/dict_cls_score.dump'
    dict_cls_score = p5_util.object_load(filename)
else :
    pass

dict_cls_score['VGG16 Seq'] = score_vgg16_seq
dict_benchmark_result = dict_cls_score.copy()

df_result = pd.DataFrame.from_dict( dict_benchmark_result, orient='index')
df_result.reset_index(inplace=True)
df_result.rename(columns={'index':'Classifier',0:'Score'}, inplace=True)
df_result
nb_images = oP7_DataBreed._sampling_breed_count*oP7_DataBreed._sampling_image_per_breed_count
nb_images = oP7_DataBreed.df_pil_image_kpdesc.shape[0]
if oP7_DataBreed.is_kp_filtered :
    title = "Benchmark classifiers accuracy / GMM clustering / "+str(nb_images)+" filtered splitted images / "+str(oP7_DataBreed.sampling_breed_count)+" breeds"
else :
    title = "Benchmark classifiers accuracy / GMM clustering / "+str(nb_images)+" splitted images / "+str(oP7_DataBreed.sampling_breed_count)+" breeds"

p6_util_plot.ser_item_occurency_plot(df_result.Classifier, df_result.Score*100, item_count=None, title=title,\
                                    p_reverse=False,p_x_title='Classifiers', p_y_title='Accuracy')

#### Classifier API
p5_util.object_dump(dict_cls_score,filename)

dict_cls_score

In [ ]:
oP7_DataBreed.show()
oP7_DataBreed.show_breed_name()